In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15, poll_frequency=1)
base_url = 'https://www.airbnb.co.in/s/Mumbai--India/homes?place_id=ChIJwe1EZjDG5zsRaYxkjY_tpF0&refinement_paths%5B%5D=%2Fhomes'
driver.get(base_url)
driver.maximize_window()

In [3]:
num_pages = 15
all_data = []

In [ ]:
for _ in range(num_pages):
    try:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.lxq01kf")))

        all_items = driver.find_elements(By.CSS_SELECTOR, "div.lxq01kf")
        if not all_items:
            break
        
        for item in all_items:
            name = item.find_element(By.CSS_SELECTOR, "div.t1jojoys").text.strip() if item.find_elements(By.CSS_SELECTOR, "div.t1jojoys") else ''
            description = item.find_element(By.CSS_SELECTOR, "span.t6mzqp7").text.strip() if item.find_elements(By.CSS_SELECTOR, "span.t6mzqp7") else ''
            rating_reviews = item.find_element(By.CSS_SELECTOR, "span.r4a59j5").text.strip() if item.find_elements(By.CSS_SELECTOR, "span.r4a59j5") else '' 

            # Handle "New" or other non-numeric values in rating and reviews
            if "out" in rating_reviews:  
                rating = rating_reviews.split(" out")[0]
                reviews = rating_reviews.split(",")[-1].strip().split(" ")[0] 
            else:
                rating = '0'
                reviews = '0' 

            price = item.find_element(By.CSS_SELECTOR, "span._11jcbg2").text.strip().replace("₹", "").strip() if item.find_elements(By.CSS_SELECTOR, "span._11jcbg2") else '' 

            # Extract place name from the name (assumes last word in name is the place)
            place = name.split()[-1] if name else ''

            all_data.append({
                'Name': name,
                'Description': description,
                'Price/Night': price,
                'Rating': rating,
                'Reviews': reviews,
                'Place': place 
            })

        # Handling pagination by clicking the "Next" button
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[aria-label='Next']")))
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            
            if 'disabled' not in next_button.get_attribute('class'):
                next_button.click()
                time.sleep(7) 
            else:
                break 

        except Exception:
            break 

    except Exception:
        break

driver.quit()


In [ ]:
df = pd.DataFrame(all_data)
df

In [ ]:
df.to_csv('airbnb_Varanasi_listings.csv', index=False, encoding='utf-8')